In [6]:
from __future__ import division, print_function, unicode_literals
import tensorflow as tf
from time import time
batch_size = 100

# Download the dataset
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.01)
    return tf.Variable(initial)
def bias_variable(shape):
    initial = tf.constant(0.0, shape=shape)
    return tf.Variable(initial)

# correct labels
y_ = tf.placeholder(tf.float32, [None, 10])
# input data
x = tf.placeholder(tf.float32, [None,28,28,1])

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [7]:
# build the network
keep_prob_input = tf.placeholder(tf.float32)
keep_prob = tf.placeholder(tf.float32)
CNW1 = weight_variable([2,2,1,64])
CNb1 = bias_variable([64])
CNW2 = weight_variable([2,2,64,32])
CNb2 = bias_variable([32])
FCW1 = weight_variable([1568,256])
FCb1 = bias_variable([256])
FCW2 = weight_variable([256,10])
FCb2 = bias_variable([10])

z = tf.nn.dropout(x, keep_prob=keep_prob_input)
z = tf.nn.conv2d(z, CNW1, [1, 1, 1, 1], padding='SAME')
z = tf.nn.relu(tf.nn.bias_add(z, CNb1))
z = tf.nn.max_pool(z, ksize=[1,2,2,1], strides=[1,2,2,1],padding='SAME')
z = tf.nn.dropout(z, keep_prob)
z = tf.nn.conv2d(z, CNW2, [1, 1, 1, 1], padding='SAME')
z = tf.nn.relu(tf.nn.bias_add(z, CNb2))
z = tf.nn.max_pool(z, ksize=[1,2,2,1], strides=[1,2,2,1],padding='SAME')
z = tf.nn.dropout(z, keep_prob)
z = tf.reshape(z, [-1,7*7*32])
z = tf.nn.relu(tf.matmul(z,FCW1) + FCb1)
z = tf.nn.dropout(z, keep_prob)
y = tf.nn.softmax(tf.matmul(z,FCW2) + FCb2)

# define the loss function
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
# define training step and accuracy
train_step = tf.train.MomentumOptimizer(learning_rate=0.1, momentum=0.9).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [13]:
# initialize the graph
init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)

# train
print("Starting the training...")
start_time = time()
best_accuracy = 0.0
for i in range(int(20*60000/batch_size)):
    input_images, correct_predictions = mnist.train.next_batch(batch_size)
    input_images = input_images.reshape(-1,28,28,1)
    if i % (60000/batch_size) == 0:
        train_accuracy = sess.run(accuracy, feed_dict={
            x: input_images, y_: correct_predictions, keep_prob_input: 1.0, keep_prob: 1.0})
        print("step %d, training accuracy %g" % (i, train_accuracy))
        # validate
        test_images = mnist.test.images.reshape(-1,28,28,1)
        test_accuracy = sess.run(accuracy, feed_dict={
            x: test_images, y_: mnist.test.labels, keep_prob_input: 1.0, keep_prob: 1.0})
        if test_accuracy >= best_accuracy:
            best_accuracy = test_accuracy
            print("Validation accuracy improved: %g." % test_accuracy)
        else:
            print("Validation accuracy was: %g. It was better before: %g. " 
                  % (test_accuracy, best_accuracy))
            break
    sess.run(train_step, feed_dict={x: input_images, y_: correct_predictions, keep_prob_input: 0.8, keep_prob: 0.5})
print("The training took %.4f seconds." % (time() - start_time))

# validate
print("Best test accuracy: %g" % best_accuracy)

Starting the training...
step 0, training accuracy 0.06
Validation accuracy improved: 0.0977.
step 600, training accuracy 0.66
Validation accuracy improved: 0.7889.
step 1200, training accuracy 0.91
Validation accuracy improved: 0.8714.
step 1800, training accuracy 0.1
Validation accuracy was: 0.098. It was better before: 0.8714. 
The training took 137.6747 seconds.
Best test accuracy: 0.8714
